<a href="https://colab.research.google.com/github/deepakjangir15/duplicate_CQA_detection/blob/main/Expermenting_Neural_Network_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'/content/drive/MyDrive/unzipped/glove.6B.200d.txt'

'/content/drive/MyDrive/unzipped/glove.6B.200d.txt'

In [2]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/unzipped/checkpoint6.pkl')

In [17]:
df.head()

,question1,question2,is_duplicate,common_words_min,common_words_max,common_stop_min,common_stop_max,common_token_min,common_token_max,last_word_equal,first_word_equal
0,Self-references in object literals / initializ...,reference variable in object literal? javascri...,1,0.428569,0.321427,0.99990,0.999900,0.303029,0.263157,0.0,0.0
1,Ubuntu and Ruby version ruby ubuntu version bu...,Installing RVM (Ruby Version Manager) ruby som...,1,0.181817,0.044444,0.00000,0.000000,0.111110,0.038461,0.0,0.0
2,Printing from a Windows Service c# printing wi...,Printing from a .NET Service c# .net windows-s...,1,0.349998,0.155555,0.99995,0.999950,0.409089,0.176470,0.0,1.0
3,Can a file be opened simultaneously under SIlv...,issue reading in data c++ fstream trouble read...,0,0.076923,0.054054,0.00000,0.000000,0.052631,0.038461,0.0,0.0
4,Nested foreach in PHP produces different resul...,Renaming project directory in VS 2010 visual-s...,0,0.047619,0.031250,0.99990,0.499975,0.068965,0.038461,0.0,0.0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398526 entries, 0 to 398525
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   question1         398526 non-null  object 
 1   question2         398526 non-null  object 
 2   is_duplicate      398526 non-null  int64  
 3   common_words_min  398526 non-null  float64
 4   common_words_max  398526 non-null  float64
 5   common_stop_min   398526 non-null  float64
 6   common_stop_max   398526 non-null  float64
 7   common_token_min  398526 non-null  float64
 8   common_token_max  398526 non-null  float64
 9   last_word_equal   398526 non-null  float64
 10  first_word_equal  398526 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 33.4+ MB


In [19]:
df.head()

,question1,question2,is_duplicate,common_words_min,common_words_max,common_stop_min,common_stop_max,common_token_min,common_token_max,last_word_equal,first_word_equal
0,Self-references in object literals / initializ...,reference variable in object literal? javascri...,1,0.428569,0.321427,0.99990,0.999900,0.303029,0.263157,0.0,0.0
1,Ubuntu and Ruby version ruby ubuntu version bu...,Installing RVM (Ruby Version Manager) ruby som...,1,0.181817,0.044444,0.00000,0.000000,0.111110,0.038461,0.0,0.0
2,Printing from a Windows Service c# printing wi...,Printing from a .NET Service c# .net windows-s...,1,0.349998,0.155555,0.99995,0.999950,0.409089,0.176470,0.0,1.0
3,Can a file be opened simultaneously under SIlv...,issue reading in data c++ fstream trouble read...,0,0.076923,0.054054,0.00000,0.000000,0.052631,0.038461,0.0,0.0
4,Nested foreach in PHP produces different resul...,Renaming project directory in VS 2010 visual-s...,0,0.047619,0.031250,0.99990,0.499975,0.068965,0.038461,0.0,0.0


In [3]:
from keras.preprocessing.text import Tokenizer
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))

In [5]:
train_q1 = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
train_q2 = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
train_y = df['is_duplicate']

In [7]:
from keras.utils import pad_sequences

train_q1 = pad_sequences(train_q1, maxlen = 30, padding='post')
train_q2 = pad_sequences(train_q2, maxlen = 30, padding='post')

In [8]:
from sklearn.model_selection import train_test_split

X_train_q1,X_valid_q1,X_train_q2, X_valid_q2, y_train, y_valid = train_test_split(train_q1, train_q2, train_y, test_size=0.20, random_state=101)

In [9]:
X_train_q1.shape

(318820, 30)

In [10]:
word_index = tokenizer.word_index

In [14]:
import numpy as np

embedding_index = {}
with open('/content/drive/MyDrive/unzipped/glove.6B.200d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [15]:
embedding_matrix = np.random.random((len(word_index)+1, 200))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [28]:
# Model for Q1
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Embedding,LSTM, Dense, Dropout, multiply, Flatten

model_q1 = tf.keras.Sequential()
model_q1.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q1.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q1.add(Dropout(0.2))
model_q1.add(LSTM(128, return_sequences = True))
model_q1.add(LSTM(128))
model_q1.add(Dense(60, activation = 'tanh'))
model_q1.add(Dense(1, activation = 'sigmoid'))

In [29]:
# Model for Q2
model_q2 = tf.keras.Sequential()
model_q2.add(Embedding(input_dim = len(word_index)+1,
                       output_dim = 200,
                      weights = [embedding_matrix],
                      input_length = 30))
model_q2.add(LSTM(128, activation = 'tanh', return_sequences = True))
model_q2.add(Dropout(0.2))
model_q2.add(LSTM(128, return_sequences = True))
model_q2.add(LSTM(128))
model_q2.add(Dense(60, activation = 'tanh'))
model_q2.add(Dense(1, activation = 'sigmoid'))

In [33]:
# Merging the output of the two models,i.e, model_q1 and model_q2
mergedOut = multiply([model_q1.output, model_q2.output])

mergedOut = Flatten()(mergedOut)
mergedOut = Dense(100, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(50, activation = 'relu')(mergedOut)
mergedOut = Dropout(0.2)(mergedOut)
mergedOut = Dense(1, activation = 'sigmoid')(mergedOut)

In [34]:
lstm_model = tf.keras.Model([model_q1.input, model_q2.input], mergedOut)
lstm_model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])


history = lstm_model.fit([X_train_q1,X_train_q2],y_train, batch_size = 2000, epochs = 10, validation_data=([X_valid_q1,X_valid_q2],y_valid))

Epoch 1/10
160/160 [==============================] - 40s 182ms/step - loss: 0.6484 - accuracy: 0.6266 - val_loss: 0.6028 - val_accuracy: 0.6925
Epoch 2/10
160/160 [==============================] - 22s 137ms/step - loss: 0.5559 - accuracy: 0.7206 - val_loss: 0.5353 - val_accuracy: 0.7304
Epoch 3/10
160/160 [==============================] - 18s 114ms/step - loss: 0.5030 - accuracy: 0.7606 - val_loss: 0.5272 - val_accuracy: 0.7422
Epoch 4/10
160/160 [==============================] - 14s 88ms/step - loss: 0.4554 - accuracy: 0.7942 - val_loss: 0.5260 - val_accuracy: 0.7441
Epoch 5/10
160/160 [==============================] - 10s 65ms/step - loss: 0.4109 - accuracy: 0.8215 - val_loss: 0.5321 - val_accuracy: 0.7462
Epoch 6/10
160/160 [==============================] - 11s 70ms/step - loss: 0.3730 - accuracy: 0.8438 - val_loss: 0.5418 - val_accuracy: 0.7499
Epoch 7/10
160/160 [==============================] - 11s 69ms/step - loss: 0.3404 - accuracy: 0.8612 - val_loss: 0.5653 - val_accura

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Dropout, LSTM, Bidirectional, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_model(vocab_size, max_length):
    input_1 = Input(shape=(max_length,))
    input_2 = Input(shape=(max_length,))

    embedding_layer = Embedding(vocab_size, 300, input_length=max_length)

    # encode question 1
    x1 = embedding_layer(input_1)
    x1 = Bidirectional(LSTM(128, return_sequences=True))(x1)
    x1 = Dropout(0.2)(x1)
    x1 = Bidirectional(LSTM(64))(x1)
    x1 = Dropout(0.2)(x1)

    # encode question 2
    x2 = embedding_layer(input_2)
    x2 = Bidirectional(LSTM(128, return_sequences=True))(x2)
    x2 = Dropout(0.2)(x2)
    x2 = Bidirectional(LSTM(64))(x2)
    x2 = Dropout(0.2)(x2)

    # concatenate encoded questions and process with dense layers
    concatenated = concatenate([x1, x2])
    dense_1 = Dense(128, activation='relu')(concatenated)
    dense_1 = Dropout(0.2)(dense_1)
    dense_2 = Dense(64, activation='relu')(dense_1)
    dense_2 = Dropout(0.2)(dense_2)
    dense_3 = Dense(32, activation='relu')(dense_2)
    dense_3 = Dropout(0.2)(dense_3)

    output = Dense(1, activation='sigmoid')(dense_3)

    model = Model(inputs=[input_1, input_2], outputs=output)
    optimizer = Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [38]:
bi_lstm_model = create_model(len(word_index)+1,30)

In [ ]:
history = bi_lstm_model.fit([X_train_q1,X_train_q2],y_train, batch_size = 2000, epochs = 10, validation_data=([X_valid_q1,X_valid_q2],y_valid))

Epoch 1/10
160/160 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.6821